## Import Required Files

In [56]:
import pandas as pd
import numpy as np
import os

In [57]:
## change it to the unzip path of the downloaded dataset..
os.chdir('/Users/rajagopalps/git/data/hackathons/MovieScriptsParticipantsData/')
data_folder = 'Scripts'

In [58]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

Total Number of Files : 2827


# Read Train Files

In [59]:
train_df = pd.read_csv('Train.csv')
train_df

,File_Name,Labels
0,file_2180.txt,8
1,file_693.txt,4
2,file_2469.txt,6
3,file_2542.txt,6
4,file_378.txt,16
...,...,...
1973,file_1930.txt,19
1974,file_1821.txt,19
1975,file_350.txt,16
1976,file_1933.txt,19


In [60]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [61]:
test_df = pd.read_csv('Test.csv')
test_df

,File_Name
0,file_2300.txt
1,file_809.txt
2,file_1383.txt
3,file_983.txt
4,file_1713.txt
...,...
844,file_2474.txt
845,file_863.txt
846,file_1547.txt
847,file_1292.txt


In [62]:
test_df.File_Name.value_counts()

file_1532.txt    1
file_499.txt     1
file_985.txt     1
file_2263.txt    1
file_1095.txt    1
                ..
file_1672.txt    1
file_530.txt     1
file_528.txt     1
file_1586.txt    1
file_971.txt     1
Name: File_Name, Length: 849, dtype: int64

In [63]:
print(train_df.Labels.nunique(), test_df.File_Name.nunique())

22 849


In [64]:
## let's read the text scripts in the train and test dataframes..

train_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

# Lets look at a script file after Reading..

In [65]:
#lets check one of the scripts..
print(train_df['Script'][4][:3000])

<b><!--

</b>if (window!= top)

top.location.href=location.href

<b>// -->

</b>

The Abyss - by James Cameron 

                                  THE ABYSS

                            AN ORIGINAL SCREENPLAY

                                      BY

                                JAMES CAMERON

                               August 2, 1988

                             Director's Revision

------------------------------------------------------------------------------

                                  THE ABYSS

OMITTED                                                                 1

OMITTED                                                                 2

TITLE: THE ABYSS -- ON BLACK, DISSOLVING TO COBALT BLUE

EXT. OCEAN/UNDERWATER -- DAY                                            3

Blue, deep and featureless, the twilight of five hundred feet down.

PROPELLER SOUND.  Materializing out of the blue limbo is the enormous but

sleek form of an Ohio-class SSBN ballistic missile su

In [66]:
print(test_df['Script'][4][:3000])

KING KONG

                                          Written by

                        Fran Walsh, Philippa Boyens and Peter Jackson

                                     Based on a Story by

                             Merian C. Cooper and Edgar Wallace

                                                                   1.

          EXT. CENTRAL PARK - DAY

          CLOSE ON: A scrawny MONKEY scratches.

          ANGLES ON: Defeated, listless ANIMALS, in the bleak environs of a

          dilapidated ZOO.

          WIDER: It is CENTRAL PARK ZOO in depression era NEW YORK. The PARK

          itself is like a GARBAGE DUMP, dotted with squalid SHANTY TOWNS.

          Against these BLEAK IMAGES, the SOUND of a BRIGHT, BRASSY SONG

          fades up: Al Jolson, singing "I'm Sitting on Top of the World".

          The sky line of MANHATTAN rises in the background, a grim steaming

          jungle on this cold FALL day.

                                                        I



In [67]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [68]:
# !pip install keras
# !pip install nltk

### There is single instance for one class (Label == 18), lets duplicate the row for stratification

In [69]:
train_df = train_df.append(train_df[train_df['Labels'] == 18])
train_df.reset_index(drop=True, inplace=True)

In [70]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
18      2
10      2
Name: Labels, dtype: int64

# Import the Modeling Libraries 

In [71]:
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


Let's load the datasets

In [72]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajagopalps/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define the Scoring Metric

In [73]:
from sklearn.metrics import log_loss

### Let's Use LabelEncoder from scikit-learn to convert Genre labels to integers, 0, 1 2

In [74]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.Labels.values)

### Before going further it is important that we split the data into training and validation sets. We can do it using 
#### `train_test_split` from the `model_selection` module of scikit-learn.

In [75]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.Script.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.30, shuffle=True)

In [76]:
print (xtrain.shape)
print (xvalid.shape)

(1385,)
(594,)


# Building Basic Models

### Let's start building our very first model. 

### Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [77]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

## TFIDF on test data

In [78]:
x_test_tfv = tfv.transform(test_df['Script'].values)

In [79]:
## Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("logloss: %0.3f " % log_loss(yvalid, predictions))

# Submission 

### Let's predict on the entire test data...

In [8]:
import pandas as pd
test_set_preds = pd.DataFrame(columns = train_df.Labels.unique().tolist())
test_set_preds.insert(0, 'File_Name', test_df.File_Name)



NameError: name 'train_df' is not defined

In [2]:
test_set_preds[test_df.Labels.unique().tolist()] = clf.predict_proba(x_test_tfv)

NameError: name 'clf' is not defined

In [3]:
test_set_preds.shape

NameError: name 'test_set_preds' is not defined

In [4]:
test_set_preds.head()

NameError: name 'test_set_preds' is not defined

In [5]:
#Reorder the columns to match the Sample_submission_file
test_set_preds = test_set_preds[['File_name',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

SyntaxError: unexpected EOF while parsing (<ipython-input-5-fce95a2772d5>, line 2)

In [6]:
#Write your submissions to an excel file
test_set_preds.to_excel('test_set_preds_v1.0.xlsx', index=False)

NameError: name 'test_set_preds' is not defined

#### Follow the sample submission format for your submission file

#### Please verify the following before submitting your solution to avoid an invalid sibmission.

1. The format of the file is excel(.xlsx)

2. The file doesn’t contain additional styling elements such as bold headings or table borders

3. The length of the submission exactly matches with that of the sample submission and test set

4. The file name doesnot have any spaces or special characters

5. All the columns are present